In [145]:
import random
import math

In [146]:
# Masked AES for d = 1 and for generic d with no ifs in its core logic (I consider I am not using any protective measures in the key expansion)

In [147]:
# 8bit random number
def rand8():
  return random.randint(0, 255)

In [148]:
# multiply by x in GF(256) :

def shift_com_r(a, r, shift):
  for i in range(shift):
    a <<= 1
    tmp = r
    a = a | tmp
  return a

def xtime(a):
  a <<= 1
  r = a & 0x100
  r >>= 8
  r = shift_com_r(r, r, 7)
  b = a ^ 0x11B
  b = b & 0xFF
  a = a & 0xFF
  ret = (r&(b ^ a)) ^ a
  return ret

In [149]:
# rotate a left n times:
def rot(a, n):
  r = a
  for i in range(n):
    r = xtime(a)
  return r

In [150]:
# a * b in GF(256) :
def gf_mul(a, b):
  res = 0
  while b:
    tmp = b & 1
    res = res  ^ (tmp*(a))
    a = xtime(a)
    b >>= 1
  return res


In [151]:
# a^e in GF(256) :
def gf_pow(a, e):
  r = 1
  while e:
    tmp = e & 1
    r = gf_mul(r, a) * tmp + (1-tmp)*r
    a = gf_mul(a, a)
    e >>= 1
  return r

In [152]:
# Simple inverse in GF(256):
def get_inv(a):
  return gf_pow(a, 254)

In [153]:
# Given p, calculate S[p] :

def rotl8(x, shift):
    return ((x << shift) | (x >> (8 - shift))) & 0xFF

def sbox(p):
    r = math.ceil(p/255)
    b = get_inv(p) & 0xFF
    tmp = (b ^
           rotl8(b,1) ^
           rotl8(b,2) ^
           rotl8(b,3) ^
           rotl8(b,4))
    return (1-r)*0x63 + r*(tmp ^ 0x63)

In [154]:
# Given a and b perform masked multiplication using the ISW algorithm with d+1 shares

def isw_mul(a, b, d):
  d = d+1
  #r = [[0]*d]*d
  #z = [[0]*d]*d
  z = [[0 for j in range(d)] for i in range(d)]
  r = [[0 for j in range(d)] for i in range(d)]
  #z = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
  c = [0]*d
  for i in range(d):
    for j in range(i+1,d):
      r[i][j] = rand8()
      z[i][j] = (r[i][j] ^ gf_mul(a[i], b[j])) ^ gf_mul(a[j], b[i])
      z[j][i] = r[i][j]
      #print(z)
  for i in range(d):
    # somatorio:
    tmp = 0
    for j in range(d):
      if i != j:
        tmp ^= z[i][j]
    c[i] = gf_mul(a[i], b[i]) ^ tmp
  #print("z : ", z)
  return c

In [155]:
# Inverse of a masked value in GF(256), working for d = 1 only :
def gf_inv_mskd(p, d):
  rand = rand8()
  b = [0x01 ^ rand, rand] # so funciona para d = 1
  e = 254
  while e:
    tmp = e & 1
    b = isw_mul(b, p, d) * tmp + (1-tmp)*b
    #print("d = 1 : ", hex(sum_shares(b, d)))
    p = isw_mul(p, p, d)
    e >>= 1
  return b

In [156]:
# AUXILIARES :
rand1 = rand8()
rand2 = rand8()
p = 0x1f
p_l = [p ^ rand1, rand1 ^ rand2, rand2]
def mask_val(p, d): # no ifs (comes with a cost of +1 random number)
  last = p
  next = rand8()
  val = [0]*(d+1)
  for i in range(d+1):
    val[i] = last ^ next
    #print("last : ", last)
    #print("next : ", next)
    #print("=========================")
    last = next
    next = rand8()
  val[d] = val[d] ^ last
  return val


def sum_shares(val, d):
  sum = 0
  for i in range(d+1):
    sum = sum ^ val[i]
  return sum


l = mask_val(79, 11)
#print(l)
print(sum_shares(l, 11))

79


In [157]:
# Trying to generalize this function:
def gf_inv_mskd_geral(p, d):
  b = mask_val(0x01, d)
  #print(b)
  e = 254
  while e: # MIMO-SNI em serie e MIMO-SIN
    tmp = e & 1
    # COLOCAR refresh p (1 input pros 2)
    b = isw_mul(b, p, d) * tmp + (1-tmp)*b
    # COLOCAR refresh b (d shares do output)
    p = isw_mul(p, p, d)
    # COLOCAR refresh p (d shares do output)
    e >>= 1
  return b

v_1 = [0x2c ^ 0x02, 0x02]
v_2 = [0x2c ^ 0x02, 0x02 ^ 0x03, 0x03]

a = [0x01 ^ 0x02, 0x02 ^ 0x03, 0x03]
b = [0x3c ^ 0x02, 0x02 ^ 0x03, 0x03]

print(hex(sum_shares(isw_mul(a, b, 2), 2)))

0x3c


In [158]:
# testar a correspondendia do gf_inv_mskd com o gf_inv_mskd_geral
for i in range(256):
  val_2 = mask_val(i, 1)
  val_3 = mask_val(i, 5)
  r_2 = gf_inv_mskd(val_2, 1)
  r_3 = gf_inv_mskd_geral(val_3, 5)
  if sum_shares(r_2, 1) != sum_shares(r_3, 5):
    print("ERRADO : \n",i)

In [159]:
# Given a masked value p, calculate S[p], only working for d = 1:

def mk_tmp_mskd(b, d):
  tmp = [0]*(d+1)
  for i in range(d+1):
    tmp[i] = b[i] ^ rotl8(b[i], 1) ^ rotl8(b[i], 2) ^ rotl8(b[i], 3) ^ rotl8(b[i], 4)
  r = 0
  return tmp

def sbox_mskd(p, d):
  r = math.ceil((p[0] ^ p[1])/255) # p e [0,255], but I must know if p == 0
  b = gf_inv_mskd(p, d)
  tmp = mk_tmp_mskd(b, d)
  tmp[0] = tmp[0]
  tmp[1] = tmp[1] ^ 0x63
  #
  r0 = rand8()
  zero_out = [0x63 ^ r0, r0]
  ret = [0, 0]
  ret[0] = (1-r)*zero_out[0] + r*(tmp[0])
  ret[1] = (1-r)*zero_out[1] + r*(tmp[1])
  return ret

In [160]:
# generalizing sbox for d
def sbox_mskd_geral(p, d):
  r = math.ceil((sum_shares(p, d))/255) # p e [0,255], but I must know if p == 0
  b = gf_inv_mskd_geral(p, d) # MIMO - SNI

  # Operacao Linear (MIMO - SNI em serie com Op linear -> MI - SNI)
  tmp = mk_tmp_mskd(b, d)
  #
  tmp[d] = tmp[d] ^ 0x63
  #
  zero_out = mask_val(0x63, d)
  ret = [0 for i in range(d+1)]
  for i in range(d+1):
    ret[i] = (1-r)*zero_out[i] + r*(tmp[i])
  #
  # Até aqui seria MI - SNI
  # COLOCAR refresh ret(d shares)
  # Até aqui seria MIMO - SNI
  return ret



In [161]:
# teste
for i in range(256):
  val_2 = mask_val(i, 1)
  val_3 = mask_val(i, 5)
  r_2 = sbox_mskd(val_2, 1)
  r_3 = sbox_mskd_geral(val_3, 5)
  if sum_shares(r_2, 1) != sum_shares(r_3, 5):
    print("ERRADO : \n",i)

In [162]:
# Mix Columns with a masked state, working for d = 1 only :
def mix_columns_mskd(state, d):
  l = len(state[0])
  tmp = [[[0, 0], [0, 0], [0, 0], [0, 0]],
         [[0, 0], [0, 0], [0, 0], [0, 0]],
         [[0, 0], [0, 0], [0, 0], [0, 0]],
         [[0, 0], [0, 0], [0, 0], [0, 0]]]
  a = [0x02 ^ 0x0, 0x0]
  b = [0x03 ^ 0x0, 0x0]
  for c in range(4):
    mul0 = isw_mul(a, state[0][c], 1)
    mul1 = isw_mul(b, state[1][c], 1)
    tmp[0][c][0] = (mul0[0] ^ mul1[0] ^ state[2][c][0] ^ state[3][c][0]) & 0xFF
    tmp[0][c][1] = (mul0[1] ^ mul1[1] ^ state[2][c][1] ^ state[3][c][1]) & 0xFF
    mul2 = isw_mul(a, state[1][c], 1)
    mul3 = isw_mul(b, state[2][c], 1)
    tmp[1][c][0] = ((state[0][c][0] ^ mul2[0]) ^ mul3[0] ^ state[3][c][0]) & 0xFF
    tmp[1][c][1] = ((state[0][c][1] ^ mul2[1]) ^ mul3[1] ^ state[3][c][1]) & 0xFF
    mul4 = isw_mul(a, state[2][c], 1)
    mul5 = isw_mul(b, state[3][c], 1)
    tmp[2][c][0] = ((state[0][c][0] ^ state[1][c][0]) ^ mul4[0] ^ mul5[0]) & 0xFF
    tmp[2][c][1] = ((state[0][c][1] ^ state[1][c][1]) ^ mul4[1] ^ mul5[1]) & 0xFF
    mul6 = isw_mul(b, state[0][c], 1)
    mul7 = isw_mul(a, state[3][c], 1)
    tmp[3][c][0] = (mul6[0] ^ state[1][c][0] ^ state[2][c][0] ^ mul7[0]) & 0xFF
    tmp[3][c][1] = (mul6[1] ^ state[1][c][1] ^ state[2][c][1] ^ mul7[1]) & 0xFF

  return tmp

In [163]:
def mix_columns_mskd_geral(state, d):
  l = len(state[0])
  tmp = [[[0 for j in range(d+1)] for i in range(4)] for n in range(4)]
  a = mask_val(0x02, d)
  b = mask_val(0x03, d)
  for c in range(4):
    for i in range(d+1):
      mul0 = isw_mul(a, state[0][c], d)
      mul1 = isw_mul(b, state[1][c], d)
      tmp[0][c][i] = (mul0[i] ^ mul1[i] ^ state[2][c][i] ^ state[3][c][i]) & 0xFF
      mul2 = isw_mul(a, state[1][c], d)
      mul3 = isw_mul(b, state[2][c], d)
      tmp[1][c][i] = ((state[0][c][i] ^ mul2[i]) ^ mul3[i] ^ state[3][c][i]) & 0xFF
      mul4 = isw_mul(a, state[2][c], d)
      mul5 = isw_mul(b, state[3][c], d)
      tmp[2][c][i] = ((state[0][c][i] ^ state[1][c][i]) ^ mul4[i] ^ mul5[i]) & 0xFF
      mul6 = isw_mul(b, state[0][c], d)
      mul7 = isw_mul(a, state[3][c], d)
      tmp[3][c][i] = (mul6[i] ^ state[1][c][i] ^ state[2][c][i] ^ mul7[i]) & 0xFF
  return tmp


In [164]:
#
def mix_columns_mskd_geral(state, d):
  l = len(state[0])
  tmp = [[[0 for j in range(d+1)] for i in range(4)] for n in range(4)]
  a = mask_val(0x02, d)
  b = mask_val(0x03, d)
  for c in range(4):
    mul0 = isw_mul(a, state[0][c], d)
    mul1 = isw_mul(b, state[1][c], d)
    #
    mul2 = isw_mul(a, state[1][c], d)
    mul3 = isw_mul(b, state[2][c], d)
    #
    mul4 = isw_mul(a, state[2][c], d)
    mul5 = isw_mul(b, state[3][c], d)
    #
    mul6 = isw_mul(b, state[0][c], d)
    mul7 = isw_mul(a, state[3][c], d)
    for i in range(d+1):
      #
      tmp[0][c][i] = (mul0[i] ^ mul1[i] ^ state[2][c][i] ^ state[3][c][i]) & 0xFF
      #
      tmp[1][c][i] = ((state[0][c][i] ^ mul2[i]) ^ mul3[i] ^ state[3][c][i]) & 0xFF
      #
      tmp[2][c][i] = ((state[0][c][i] ^ state[1][c][i]) ^ mul4[i] ^ mul5[i]) & 0xFF
      #
      tmp[3][c][i] = (mul6[i] ^ state[1][c][i] ^ state[2][c][i] ^ mul7[i]) & 0xFF
  return tmp

In [165]:
#
state =   [[[99 ^ 1, 1], [1^1, 1], [45^1, 1], [242^1, 1]],
         [[71 ^ 2, 2], [1^2, 2], [38^2, 2], [10^2, 2]],
         [[162 ^ 3, 3], [1^3, 3], [49^3, 3], [34^3, 3]],
         [[240 ^ 4, 4], [1^4, 4], [76^4, 4], [92^4, 4]]]
#print(len(v[0]))
tmp = mix_columns_mskd(state, 1)
print(state)
for i in range(4):
  #print(state[i][0][0] ^ state[i][0][1])
  for j in range(4):
    print(tmp[j][i][0] ^ tmp[j][i][1])
  print("------")

[[[98, 1], [0, 1], [44, 1], [243, 1]], [[69, 2], [3, 2], [36, 2], [8, 2]], [[161, 3], [2, 3], [50, 3], [33, 3]], [[244, 4], [5, 4], [72, 4], [88, 4]]]
93
224
112
187
------
1
1
1
1
------
77
126
189
248
------
159
220
88
157
------


In [166]:
#
state =   [[[99 ^ 1, 1 ^ 2, 2], [1^1, 1 ^ 2, 2], [45^1, 1 ^ 2, 2], [242^1, 1 ^ 2, 2]],
         [[71 ^ 2, 2 ^ 3, 3], [1^2, 2 ^ 3, 3], [38^2, 2 ^ 3, 3], [10^2, 2 ^ 3, 3]],
         [[162 ^ 3, 3 ^ 4, 4], [1^3, 3 ^ 4, 4], [49^3, 3 ^ 4, 4], [34^3, 3 ^ 4, 4]],
         [[240 ^ 4, 4 ^ 5, 5], [1^4, 4 ^ 5, 5], [76^4, 4 ^ 5, 5], [92^4, 4 ^ 5, 5]]]
#print(len(v[0]))
tmp = mix_columns_mskd_geral(state, 2)
#print(state)
for i in range(4):
  #print(state[i][0][0] ^ state[i][0][1])
  for j in range(4):
    #print(tmp[j][i][0] ^ tmp[j][i][1])
    print(sum_shares(tmp[j][i], 2))
  print("------")

93
224
112
187
------
1
1
1
1
------
77
126
189
248
------
159
220
88
157
------


In [167]:
# Shift Rows(It does not matter with state is masked or not) :
def shiftrows(state):
    assert len(state) == 4 and all(len(row) == 4 for row in state)
    # Row 0: no shift
    state[1] = state[1][1:] + state[1][:1]
    state[2] = state[2][2:] + state[2][:2]
    state[3] = state[3][3:] + state[3][:3]

    return state

In [168]:
# Given a masked state and round_key, perform an xor term by term :
def add_round_key_mskd(state, round_key, d):
    tmp = state.copy()
    for k in range(2):
      for i in range(4):
        for j in range(4):
          tmp[i][j][k] ^= round_key[i][j][k]
    return tmp

In [169]:
###################################### BEGIN KEY SCHEDULE ######################################################
#

RCON = [
    0x00, # unused
    0x01, 0x02, 0x04, 0x08,
    0x10, 0x20, 0x40, 0x80,
    0x1b, 0x36
]

def rot_word(word):
    return word[1:] + word[:1]

def sub_word(word):
    return [sbox(b) for b in word]

def key_exp_core(word, i):
    word = rot_word(word)
    word = sub_word(word)
    word[0] ^= gf_pow(0x02, i-1) # rcon(i) = gf_pow(2, i-1)
    return word

def key_exp(key):
    # key: list of 16 bytes
    assert len(key) == 16
    expanded = [0] * 176
    expanded[0:16] = key

    bytes_generated = 16
    rcon_iter = 1
    temp = [0, 0, 0, 0]

    while bytes_generated < 176:
        # last 4 bytes
        temp = expanded[bytes_generated-4:bytes_generated]

        if bytes_generated % 16 == 0:
            temp = key_exp_core(temp, rcon_iter)
            rcon_iter += 1

        for i in range(4):
            expanded[bytes_generated] = expanded[bytes_generated - 16] ^ temp[i]
            bytes_generated += 1

    return expanded


key = [0x0]*16
#
def mask_exp_keys(expd_keys):
  #
  m = rand8()
  tmp = [[0] for _ in range(176)]

  for i in range(len(expd_keys)):
    tmp[i] = [expd_keys[i] ^ m, m] # only working for d = 1
  return tmp

print(mask_exp_keys(key))
#
def mask_exp_keys_geral(expd_keys, d):
  #
  m = rand8()
  tmp = [[0] for _ in range(176)]

  for i in range(len(expd_keys)):
    #tmp[i] = [expd_keys[i] ^ m, m] # only working for d = 1
    tmp[i] = [expd_keys[i], 0, 0]
    #tmp[i] = mask_val(expd_keys[i], d)
  return tmp

print(mask_exp_keys_geral(key, 2))


def test_key_exp():
    key = [
        0x2b, 0x7e, 0x15, 0x16,
        0x28, 0xae, 0xd2, 0xa6,
        0xab, 0xf7, 0x15, 0x88,
        0x09, 0xcf, 0x4f, 0x3c
    ]
    expanded = key_exp(key)
    assert len(expanded) == 176

    expected_r1 = [
        0xa0,0xfa,0xfe,0x17, 0x88,0x54,0x2c,0xb1,
        0x23,0xa3,0x39,0x39, 0x2a,0x6c,0x76,0x05
    ]
    expected_r2 = [
        0xf2,0xc2,0x95,0xf2, 0x7a,0x96,0xb9,0x43,
        0x59,0x35,0x80,0x7a, 0x73,0x59,0xf6,0x7f
    ]
    expected_r3 = [
        0x3d,0x80,0x47,0x7d, 0x47,0x16,0xfe,0x3e,
        0x1e,0x23,0x7e,0x44, 0x6d,0x7a,0x88,0x3b
    ]

    assert expanded[16:32] == expected_r1, "Round 1 mismatch"
    assert expanded[32:48] == expected_r2, "Round 2 mismatch"
    assert expanded[48:64] == expected_r3, "Round 3 mismatch"

    print("AES-128 key expansion matches FIPS-197 Appendix A.1 up to round 3!")
test_key_exp()


[[56, 56], [56, 56], [56, 56], [56, 56], [56, 56], [56, 56], [56, 56], [56, 56], [56, 56], [56, 56], [56, 56], [56, 56], [56, 56], [56, 56], [56, 56], [56, 56], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]
[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0

In [170]:
# For each value in a masked state, get the corresponding sbox value :
def sub_bytes_masked(state, d):
  for i in range(4):
    for j in range(4):
      state[i][j] = sbox_mskd(state[i][j], d)
  return state

In [171]:
# Masked AES for d = 1
def masked_aes(text, keys, d):
  state = mk_matrix_2(text)
  state = add_round_key_mskd(state, mk_matrix_2(keys[0:16]), d)
  for i in range(1, 10):
    state = sub_bytes_masked(state, d)
    state = shiftrows(state)
    state = mix_columns_mskd(state, d)
    state = add_round_key_mskd(state, mk_matrix_2(keys[i*16:i*16+16]), d)
  state = sub_bytes_masked(state, d)
  state = shiftrows(state)
  state = add_round_key_mskd(state, mk_matrix_2(keys[160:176]), d)
  return state

In [172]:
# The operations of this AES implementation are done in matrixes, so if an input is given in an list
# I must parse it to a matrix in column major:
def mk_matrix_2(l):
    tmp = [[0]*4 for _ in range(4)]
    k = 0
    for c in range(4):       # columns first
        for r in range(4):   # then rows
            tmp[r][c] = l[k]
            k += 1
    return tmp


In [173]:
# TEST 1 From AESAVS.doc: 3ad78e726c1ec02b7ebfe92b23d9ec34
key = [0x0]*16
exp_key = key_exp(key)
exp_key = mask_exp_keys(exp_key)

text = [[0x80 ^ 0x1, 0x1], [0x0 ^ 0x1, 0x1], [0x0 ^ 0x1, 0x1], [0x0 ^ 0x1, 0x1],
       [0x0 ^ 0x2, 0x2], [0x0 ^ 0x2, 0x2], [0x0 ^ 0x2, 0x2], [0x0 ^ 0x2, 0x2],
       [0x0 ^ 0x3, 0x3], [0x0 ^ 0x3, 0x3], [0x0 ^ 0x3, 0x3], [0x0 ^ 0x3, 0x3],
       [0x0 ^ 0x4, 0x4], [0x0 ^ 0x4, 0x4], [0x0 ^ 0x4, 0x4], [0x0 ^ 0x4, 0x4]]

cipher = masked_aes(text, exp_key, 1)
out = []
for i in range(4):
    for j in range(4):
        out.append(cipher[j][i][0] ^ cipher[j][i][1])  # careful with order
print("Ciphertext:", "".join(f"{b:02x}" for b in out))

Ciphertext: 3ad78e726c1ec02b7ebfe92b23d9ec34


In [174]:
# TEST 2 From AESAVS.doc: 79bf5dce14bb7dd73a8e3611de7ce026
key = [0x00]*16
exp_key = key_exp(key)

text = [[0xff ^ 0x1, 0x1], [0xff ^ 0x1, 0x1], [0xff ^ 0x1, 0x1], [0xff ^ 0x1, 0x1],
        [0xff ^ 0x2, 0x2], [0xff ^ 0x2, 0x2], [0xff ^ 0x2, 0x2], [0xff ^ 0x2, 0x2],
        [0xff ^ 0x3, 0x3], [0xff ^ 0x3, 0x3], [0xff ^ 0x3, 0x3], [0xff ^ 0x3, 0x3],
        [0xfc ^ 0x4, 0x4], [0x0 ^ 0x4, 0x4], [0x0 ^ 0x4, 0x4], [0x0 ^ 0x4, 0x4]]
exp_key = mask_exp_keys(exp_key)
cipher = masked_aes(text, exp_key, 1)
out = []
for i in range(4):
    for j in range(4):
        out.append(cipher[j][i][0] ^ cipher[j][i][1])  # careful with order
print("Ciphertext:", "".join(f"{b:02x}" for b in out))

Ciphertext: 79bf5dce14bb7dd73a8e3611de7ce026


In [175]:
# TEST 3 From AESAVS.doc: 6d251e6944b051e04eaa6fb4dbf78465
key = [0x10, 0xa5, 0x88, 0x69, 0xd7, 0x4b, 0xe5, 0xa3,
           0x74, 0xcf, 0x86, 0x7c, 0xfb, 0x47, 0x38, 0x59]
exp_key = key_exp(key)

text = [[0x0 ^ 0x1, 0x1], [0x0 ^ 0x1, 0x1], [0x0 ^ 0x1, 0x1], [0x0 ^ 0x1, 0x1],
        [0x0 ^ 0x2, 0x2], [0x0 ^ 0x2, 0x2], [0x0 ^ 0x2, 0x2], [0x0 ^ 0x2, 0x2],
        [0x0 ^ 0x3, 0x3], [0x0 ^ 0x3, 0x3], [0x0 ^ 0x3, 0x3], [0x0 ^ 0x3, 0x3],
        [0x0 ^ 0x4, 0x4], [0x0 ^ 0x4, 0x4], [0x0 ^ 0x4, 0x4], [0x0 ^ 0x4, 0x4]]
exp_key = mask_exp_keys(exp_key)
cipher = masked_aes(text, exp_key, 1)
out = []
for i in range(4):
    for j in range(4):
        out.append(cipher[j][i][0] ^ cipher[j][i][1])  # careful with order
print("Ciphertext:", "".join(f"{b:02x}" for b in out))

Ciphertext: 6d251e6944b051e04eaa6fb4dbf78465


In [176]:
# Testing for using more shares

# For each value in a masked state, get the corresponding sbox value :
def sub_bytes_masked_geral(state, d):
  for i in range(4):
    for j in range(4):
      state[i][j] = sbox_mskd_geral(state[i][j], d)
  return state


# Masked AES
def masked_aes_geral(text, keys, d):
  state = mk_matrix_2(text)
  state = add_round_key_mskd(state, mk_matrix_2(keys[0:16]), d)
  for i in range(1, 10):
    state = sub_bytes_masked_geral(state, d)
    state = shiftrows(state)
    state = mix_columns_mskd_geral(state, d)
    state = add_round_key_mskd(state, mk_matrix_2(keys[i*16:i*16+16]), d)
  state = sub_bytes_masked_geral(state, d)
  state = shiftrows(state)
  state = add_round_key_mskd(state, mk_matrix_2(keys[160:176]), d)
  return state


In [177]:
####################### Test for general case

In [178]:
# TEST 3 From AESAVS.doc: 6d251e6944b051e04eaa6fb4dbf78465
key = [0x10, 0xa5, 0x88, 0x69, 0xd7, 0x4b, 0xe5, 0xa3,
           0x74, 0xcf, 0x86, 0x7c, 0xfb, 0x47, 0x38, 0x59]
exp_key = key_exp(key)

text = [[0x0 ^ 0x1, 0x1 ^ 0x2, 0x2], [0x0 ^ 0x1, 0x1 ^ 0x2, 0x2], [0x0 ^ 0x1, 0x1 ^ 0x2, 0x2], [0x0 ^ 0x1, 0x1 ^ 0x2, 0x2],
        [0x0 ^ 0x2, 0x2 ^ 0x3, 0x3], [0x0 ^ 0x2, 0x2 ^ 0x3, 0x3], [0x0 ^ 0x2, 0x2 ^ 0x3, 0x3], [0x0 ^ 0x2, 0x2 ^ 0x3, 0x3],
        [0x0 ^ 0x3, 0x3 ^ 0x4, 0x4], [0x0 ^ 0x3, 0x3 ^ 0x4, 0x4], [0x0 ^ 0x3, 0x3 ^ 0x4, 0x4], [0x0 ^ 0x3, 0x3 ^ 0x4, 0x4],
        [0x0 ^ 0x4, 0x4 ^ 0x5, 0x5], [0x0 ^ 0x4, 0x4 ^ 0x5, 0x5], [0x0 ^ 0x4, 0x4 ^ 0x5, 0x5], [0x0 ^ 0x4, 0x4 ^ 0x5, 0x5]]

state = mk_matrix_2(text)
for i in range(4):
  #print(state[i][0][0] ^ state[i][0][1])
  for j in range(4):
    #print(tmp[j][i][0] ^ tmp[j][i][1])
    #print(text[j][i])
    print(sum_shares(state[j][i], 2))
  print("------")

exp_key = mask_exp_keys_geral(exp_key, 2)
cipher = masked_aes_geral(text, exp_key, 2)
out = []
for i in range(4):
    for j in range(4):
        #out.append(cipher[j][i][0] ^ cipher[j][i][1])  # careful with order
        out.append(sum_shares(cipher[j][i], 2))
print("Ciphertext:", "".join(f"{b:02x}" for b in out))

0
0
0
0
------
0
0
0
0
------
0
0
0
0
------
0
0
0
0
------
Ciphertext: 6d251e6944b051e04eaa6fb4dbf78465


In [179]:
# TEST 2 From AESAVS.doc: 79bf5dce14bb7dd73a8e3611de7ce026
key = [0x00]*16
exp_key = key_exp(key)

text = [[0xff ^ 0x1, 0x1 ^ 0x02, 0x02], [0xff ^ 0x1, 0x1 ^ 0x02, 0x02], [0xff ^ 0x1, 0x1 ^ 0x02, 0x02], [0xff ^ 0x1, 0x1 ^ 0x02, 0x02],
        [0xff ^ 0x2, 0x2 ^ 0x03, 0x03], [0xff ^ 0x2, 0x2 ^ 0x03, 0x03], [0xff ^ 0x2, 0x2 ^ 0x03, 0x03], [0xff ^ 0x2, 0x2 ^ 0x03, 0x03],
        [0xff ^ 0x3, 0x3 ^ 0x04, 0x04], [0xff ^ 0x3, 0x3 ^ 0x04, 0x04], [0xff ^ 0x3, 0x3 ^ 0x04, 0x04], [0xff ^ 0x3, 0x3 ^ 0x04, 0x04],
        [0xfc ^ 0x4, 0x4 ^ 0x05, 0x05], [0x0 ^ 0x4, 0x4 ^ 0x05, 0x05], [0x0 ^ 0x4, 0x4 ^ 0x05, 0x05], [0x0 ^ 0x4, 0x4 ^ 0x05, 0x05]]

exp_key = mask_exp_keys_geral(exp_key, 2)
cipher = masked_aes_geral(text, exp_key, 2)
out = []
for i in range(4):
    for j in range(4):
        #out.append(cipher[j][i][0] ^ cipher[j][i][1])  # careful with order
        out.append(sum_shares(cipher[j][i], 2))
print("Ciphertext:", "".join(f"{b:02x}" for b in out))

Ciphertext: 79bf5dce14bb7dd73a8e3611de7ce026


In [180]:
# TEST 1 From AESAVS.doc: 3ad78e726c1ec02b7ebfe92b23d9ec34
key = [0x0]*16
exp_key = key_exp(key)

text = [[0x80 ^ 0x1, 0x1 ^ 0x2, 0x02], [0x0 ^ 0x1, 0x1 ^ 0x2, 0x02], [0x0 ^ 0x1, 0x1 ^ 0x2, 0x02], [0x0 ^ 0x1, 0x1 ^ 0x2, 0x02],
       [0x0 ^ 0x2, 0x2 ^ 0x3, 0x3], [0x0 ^ 0x2, 0x2 ^ 0x3, 0x3], [0x0 ^ 0x2, 0x2 ^ 0x3, 0x3], [0x0 ^ 0x2, 0x2 ^ 0x3, 0x3],
       [0x0 ^ 0x3, 0x3 ^ 0x4, 0x4], [0x0 ^ 0x3, 0x3 ^ 0x4, 0x4], [0x0 ^ 0x3, 0x3 ^ 0x4, 0x4], [0x0 ^ 0x3, 0x3 ^ 0x4, 0x4],
       [0x0 ^ 0x4, 0x4 ^ 0x5, 0x5], [0x0 ^ 0x4, 0x4 ^ 0x5, 0x5], [0x0 ^ 0x4, 0x4 ^ 0x5, 0x5], [0x0 ^ 0x4, 0x4 ^ 0x5, 0x5]]

exp_key = mask_exp_keys_geral(exp_key, 2)
cipher = masked_aes_geral(text, exp_key, 2)
out = []
for i in range(4):
    for j in range(4):
        #out.append(cipher[j][i][0] ^ cipher[j][i][1])  # careful with order
        out.append(sum_shares(cipher[j][i], 2))
print("Ciphertext:", "".join(f"{b:02x}" for b in out))

Ciphertext: 3ad78e726c1ec02b7ebfe92b23d9ec34
